# Создание onnx из .pt файла

### Установка зависимостей

Установите зависимости для YoLo

Сначала для torch.cpu:
https://github.com/ValV/yolov7/blob/master/requirements.torch.cpu.txt

Остальные:
https://github.com/ValV/yolov7/blob/master/requirements.txt


Установка самой библиотеки:

In [1]:
%pip install -U git+https://github.com/ValV/yolov7.git@package#egg=yolov7

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/ValV/yolov7.git (to revision package) to c:\users\user\appdata\local\temp\pip-install-2sefd0ej\yolov7_595d265ee7b448c692765954c75299a2
  Resolved https://github.com/ValV/yolov7.git to commit f7d72f0b0d41f996d8a21268c05bc29f23d97196
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for yolov7: filename=yolov7-0.1.0-py3-none-any.whl size=132407 sha256=e52a59ffc7b59a9877e11ff710aa794430c635f56de3e3b1d28044376cd40b28
  Stored in directory: C:\Users\user\AppData\Local\Temp\pip-ephem-wheel-cache-crt6n5ce\wheels\db\d1\c7\e5d94140ce224fd3346cf691d55dee88bb5d22ba6670d68fd2
Successfully built yolov7
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/ValV/yolov7.git 'C:\Users\user\AppData\Local\Temp\pip-install-2sefd0ej\yolov7_595d265ee7b448c692765954c75299a2'
  Running command git checkout -b package --track origin/package
  branch 'package' set up to track 'origin/package'.
  Switched to a new branch 'package'


Установка библиотек для взаимодействия с onnx

In [2]:
%pip install 'nvidia-pyindex'
%pip install 'onnx>=1.9.0' 'onnx-simplifier>=0.3.6' 'onnx-graphsurgeon' 'protobuf~=3.19.6'

Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: "'nvidia-pyindex'"


Note: you may need to restart the kernel to use updated packages.


ERROR: Invalid requirement: "'onnx"


### Код

Импорт библиотек

In [5]:
import time

from dataclasses import dataclass
from os import path as osp

In [6]:
import logging
import math
import os
import random
import sys
import time
from copy import deepcopy
from pathlib import Path
from threading import Thread

import numpy as np
import torch.distributed as dist
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
import torch.utils.data
import yaml
from torch.cuda import amp
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

import warnings


In [7]:
from yolov7.models.common import Conv
from yolov7.models.experimental import attempt_load, End2End
from yolov7.utils.activations import Hardswish, SiLU
from yolov7.utils.general import set_logging, check_img_size
from yolov7.utils.torch_utils import select_device
from yolov7.utils.add_nms import RegisterNMS


from os import makedirs
from glob import glob

Функция для импорта

In [8]:




def export(
    weights: str,  # weights path
    img_size: int = [640, 640],  # image size
    batch_size: int = 1,  # batch size
    dynamic: bool = False,  # dynamic ONNX axes
    dynamic_batch: bool = False,  # dynamic batch onnx for tensorrt and onnx-runtime (disables dynamic axes)
    grid: bool = False,  # export Detect() layer grid
    end2end: bool = False,  # export end2end onnx (disables dynamic axes)
    max_wh: int = None,  # None for tensorrt nms, int value for onnx-runtime nms
    topk_all: int = 100,  # topk objects for every image
    iou_thres: float = 0.45,  # iou threshold for NMS
    conf_thres: float = 0.25,  # conf threshold for NMS
    device: str = 'cpu',  # cuda device, i.e. 0 or 0,1,2,3 or cpu
    simplify: bool = False,  # simplify onnx model
    include_nms: bool = False,  # export end2end onnx
    fp16: bool = False,  # CoreML FP16 half-precision export
    int8: bool = False  # CoreML INT8 quantization
):
    img_size *= 2 if len(img_size) == 1 else 1  # expand
    dynamic = dynamic and not end2end
    dynamic = False if dynamic_batch else dynamic
    set_logging()
    t = time.time()

    # Load PyTorch model
    device = select_device(device)
    model = attempt_load(weights, map_location=device)  # load FP32 model
    labels = model.names

    # Checks
    gs = int(max(model.stride))  # grid size (max stride)
    img_size = [check_img_size(x, gs) for x in img_size]  # verify img_size are gs-multiples

    # Input
    img = torch.zeros(batch_size, 3, *img_size).to(device)  # image size(1, 3, 320, 192) iDetection

    # Update model
    for k, m in model.named_modules():
        m._non_persistent_buffers_set = set()  # pytorch 1.6.0 compatibility
        if isinstance(m, Conv):  # assign export-friendly activations
            if isinstance(m.act, nn.Hardswish):
                m.act = Hardswish()
            elif isinstance(m.act, nn.SiLU):
                m.act = SiLU()
        # elif isinstance(m, models.yolo.Detect):
        #     m.forward = m.forward_export  # assign forward (optional)
    model.model[-1].export = not grid  # set Detect() layer grid export
    y = model(img)  # dry run
    if include_nms:
        model.model[-1].include_nms = True
        y = None

    # TorchScript export
    try:
        print('\nStarting TorchScript export with torch %s...' % torch.__version__)
        f = weights.replace('.pt', '.torchscript.pt')  # filename
        ts = torch.jit.trace(model, img, strict=False)
        ts.save(f)
        print('TorchScript export success, saved as %s' % f)
    except Exception as e:
        print('TorchScript export failure: %s' % e)

    # CoreML export
    try:
        import coremltools as ct

        print('\nStarting CoreML export with coremltools %s...' % ct.__version__)
        # convert model from torchscript and apply pixel scaling as per detect.py
        ct_model = ct.convert(ts, inputs=[ct.ImageType('image', shape=img.shape, scale=1 / 255.0, bias=[0, 0, 0])])
        bits, mode = (8, 'kmeans_lut') if int8 else (16, 'linear') if fp16 else (32, None)
        if bits < 32:
            if sys.platform.lower() == 'darwin':  # quantization only supported on macOS
                with warnings.catch_warnings():
                    warnings.filterwarnings("ignore", category=DeprecationWarning)  # suppress numpy==1.20 float warning
                    ct_model = ct.models.neural_network.quantization_utils.quantize_weights(ct_model, bits, mode)
            else:
                print('quantization only supported on macOS, skipping...')

        f = weights.replace('.pt', '.mlmodel')  # filename
        ct_model.save(f)
        print('CoreML export success, saved as %s' % f)
    except Exception as e:
        print('CoreML export failure: %s' % e)

    # TorchScript-Lite export
    try:
        print('\nStarting TorchScript-Lite export with torch %s...' % torch.__version__)
        f = weights.replace('.pt', '.torchscript.ptl')  # filename
        tsl = torch.jit.trace(model, img, strict=False)
        tsl = optimize_for_mobile(tsl)
        tsl._save_for_lite_interpreter(f)
        print('TorchScript-Lite export success, saved as %s' % f)
    except Exception as e:
        print('TorchScript-Lite export failure: %s' % e)

    # ONNX export
    try:
        import onnx

        print('\nStarting ONNX export with onnx %s...' % onnx.__version__)
        f = weights.replace('.pt', '.onnx')  # filename
        model.eval()
        output_names = ['classes', 'boxes'] if y is None else ['output']
        dynamic_axes = None
        if dynamic:
            dynamic_axes = {
                'images': {0: 'batch', 2: 'height', 3: 'width'},  # size(1, 3, 640, 640)
                'output': {0: 'batch', 2: 'y', 3: 'x'}
            }
        if dynamic_batch:
            batch_size = 'batch'
            dynamic_axes = {
                'images': {
                    0: 'batch',
                },
            }
            if end2end and max_wh is None:
                # TensorRT end2end
                output_axes = {
                    'num_dets': {0: 'batch'},
                    'det_boxes': {0: 'batch'},
                    'det_scores': {0: 'batch'},
                    'det_classes': {0: 'batch'},
                }
            else:
                # Onnxruntime
                output_axes = {
                    'output': {0: 'batch'},
                }
            dynamic_axes.update(output_axes)
        if grid:
            if end2end:
                # End2end Detect() layer grid export
                print('\nStarting export end2end onnx model for %s...' % 'TensorRT' if max_wh is None else 'onnxruntime')
                model = End2End(model, topk_all, iou_thres, conf_thres, max_wh, device, len(labels))
                if end2end and max_wh is None:
                    # TensorRT end2end
                    output_names = ['num_dets', 'det_boxes', 'det_scores', 'det_classes']
                    shapes = [batch_size, 1, batch_size, topk_all, 4,
                              batch_size, topk_all, batch_size, topk_all]
                else:
                    # Onnxruntime end2end
                    output_names = ['output']
            else:
                # Basic Detect() layer grid export
                model.model[-1].concat = True

        torch.onnx.export(
            model, img, f, verbose=False, opset_version=12,
            input_names=['images'],
            output_names=output_names,
            dynamic_axes=dynamic_axes
        )

        # Checks
        onnx_model = onnx.load(f)  # load onnx model
        onnx.checker.check_model(onnx_model)  # check onnx model

        if end2end and max_wh is None:
            # TensorRT end2end
            for i in onnx_model.graph.output:
                for j in i.type.tensor_type.shape.dim:
                    j.dim_param = str(shapes.pop(0))

        # print(onnx.helper.printable_graph(onnx_model.graph))  # print a human readable model

        # # Metadata
        # d = {'stride': int(max(model.stride))}
        # for k, v in d.items():
        #     meta = onnx_model.metadata_props.add()
        #     meta.key, meta.value = k, str(v)
        # onnx.save(onnx_model, f)

        if simplify:
            try:
                import onnxsim

                print('\nStarting to simplify ONNX...')
                onnx_model, check = onnxsim.simplify(onnx_model)
                assert check, 'assert check failed'
            except Exception as e:
                print(f'Simplifier failure: {e}')

        # print(onnx.helper.printable_graph(onnx_model.graph))  # print a human readable model
        onnx.save(onnx_model, f)
        print('ONNX export success, saved as %s' % f)

        if include_nms:
            print('Registering NMS plugin for ONNX...')
            mo = RegisterNMS(f)
            # if cuda is available -- TRT
            # else something without TRT 
            mo.register_nms()
            mo.save(f)

    except Exception as e:
        raise
        print('ONNX export failure: %s' % e)

    # Finish
    print('\nExport complete (%.2fs). Visualize with https://github.com/lutzroeder/netron.' % (time.time() - t))
    
    


Код, который запускает сборку onnx

In [9]:
@dataclass
class Config: ...


opt = Config()
opt.save_dir = '.'


path_export_source = osp.join(opt.save_dir, 'weights')
# path_export_target = osp.join(opt.save_dir, 'models')

# makedirs(path_export_target, exist_ok=True)

size_input = [3840, 2176]  # [3840, 2160] (4K) + multiple of 32
for path_weights in glob(osp.join(path_export_source, '????.pt')):
    if 'init.pt' in path_weights:
        continue
    print(f"Exporting {osp.basename(path_weights)}...")
    export(
        weights=path_weights,
        img_size=size_input,  # opt.img_size,
        dynamic=True,
        # dynamic_batch=True,
        # end2end=True,
        # max_wh=size_input,
        simplify=True,
        include_nms=False
    )
    print('\n\n')

YOLOR  2024-4-9 torch 2.2.2+cpu CPU



Exporting best.pt...
Fusing layers... 


c:\Users\user\miniconda3\envs\project-p\lib\site-packages\torch\functional.py:507: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3550.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Model Summary: 200 layers, 6017434 parameters, 0 gradients, 13.1 GFLOPS



Starting TorchScript export with torch 2.2.2+cpu...
TorchScript export success, saved as .\weights\best.torchscript.pt
CoreML export failure: No module named 'coremltools'

Starting TorchScript-Lite export with torch 2.2.2+cpu...
TorchScript-Lite export failure: name 'optimize_for_mobile' is not defined

Starting ONNX export with onnx 1.12.0...


c:\Users\user\miniconda3\envs\project-p\lib\site-packages\yolov7\models\yolo.py:590: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if augment:
c:\Users\user\miniconda3\envs\project-p\lib\site-packages\yolov7\models\yolo.py:622: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if profile:
c:\Users\user\miniconda3\envs\project-p\lib\site-packages\yolov7\models\yolo.py:637: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant i


Starting to simplify ONNX...
ONNX export success, saved as .\weights\best.onnx

Export complete (75.34s). Visualize with https://github.com/lutzroeder/netron.



Exporting last.pt...
Fusing layers... 


Model Summary: 200 layers, 6017434 parameters, 0 gradients, 13.1 GFLOPS



Starting TorchScript export with torch 2.2.2+cpu...
TorchScript export success, saved as .\weights\last.torchscript.pt
CoreML export failure: No module named 'coremltools'

Starting TorchScript-Lite export with torch 2.2.2+cpu...
TorchScript-Lite export failure: name 'optimize_for_mobile' is not defined

Starting ONNX export with onnx 1.12.0...

Starting to simplify ONNX...
ONNX export success, saved as .\weights\last.onnx

Export complete (82.39s). Visualize with https://github.com/lutzroeder/netron.



